In [1]:
using NetCDF
using Printf

### specify files

In [6]:
## specify location of .nc file
ncfile = "/home/kbayliss/gem-hazard-data/vs30/global_vs30.grd";

## specify location of "sites_csv" file (lon, lat)
fname_in = "/home/kbayliss/gem/mosaic/WAF/sites_test_h6.csv"

## specify location of output "site_model_file" file (lon,lat,vs30,z1pt0,z2pt5,vs30measured)
fname_out = "/home/kbayliss/gem/mosaic/WAF/out.csv"

"/home/kbayliss/gem/mosaic/WAF/out.csv"

### create functions

In [22]:
function get_index(vec, value)
    dff = vec .- value
    return findmin(abs.(dff))
end

function get_sites(ncfile)
    lons = Float64[]
    lats = Float64[]
    file=open(ncfile, "r")
    c = 0
    while(!eof(file))
        line = readline(file)
        svec = split(line, ",")
        if !all(isletter, svec[1])
            push!(lons, parse(Float64, svec[1]))
            push!(lats, parse(Float64, svec[2])) 
            c += 1
        end
    end
    println(c)
    return lons, lats
end

function get_vs30(slons, slats, lons, lats, vs30)
    vals = Float64[]
    for (lo, la) in zip(slons, slats)
        idxx = get_index(xs, lo)
        idxy = get_index(ys, la)
        push!(vals, zs[idxx[2], idxy[2]])
    end
    return vals
end

# For 2026, z1pt0 and z2pt5 should be 0
function get_z1pt0(vs30)
    c1 = 571^4.
    c2 = 1360.0^4.
    out = exp.((-7.15 / 4.0) * log.((vs30.^4. .+ c1) ./ (c2 + c1)))
    return out
end

function get_z2pt5(vs30)
    c1 = 7.089
    c2 = -1.144
    out = exp.(c1 .+ log.(vs30) .* c2)
    return out
end

      
function write_site_file(ncfile, lons, lats, vs30)
    z1pt0 = repeat([-999], size(lons)[1])
    z2pt5 = repeat([-999], size(lons)[1])
    f = open(ncfile, "w")
    write(f, "lon,lat,vs30,z1pt0,z2pt5,vs30measured\n")
    c = 0
    for (lo, la, vs, z1, z2) in zip(lons, lats, vs30, z1pt0, z2pt5)
        tmp = @sprintf("%.6f,%.6f,%.2f,%.2f,%.2f,%d\n", lo, la, vs, z1, z2, 1)
        write(f, tmp)
        c += 1
    end
    close(f)
    println(c)
end

write_site_file (generic function with 1 method)

### read in .nc file

In [4]:
info = ncinfo(ncfile)
println(info)

xs = ncread(ncfile, "x");
ys = ncread(ncfile, "y");
zs = ncread(ncfile, "z"); #the vs30 field


##### NetCDF File #####

/home/kbayliss/gem-hazard-data/vs30/global_vs30.grd

##### Dimensions #####

Name                                                Length                    
--------------------------------------------------------------------------------
x                                                   43201                     
y                                                   16801                     

##### Variables #####

Name                            Type            Dimensions                      
--------------------------------------------------------------------------------
x                               DOUBLE          x                               
z                               FLOAT           x y                             
y                               DOUBLE          y                               

##### Attributes #####

Variable            Name                Value                                   
--------------------------------------------

### read in sites and get vs30

In [7]:
lons, lats = get_sites(fname_in)
vs30 = get_vs30(lons, lats, xs, ys, zs);

6329


In [11]:
size(vs30)

(6329,)

### save file

In [23]:
write_site_file(fname_out, lons, lats, vs30)

6329
